\begin{equation}
    k_B = J = 1
\end{equation}

In [1]:
using Pkg
Pkg.activate("./Packages/Project.toml");

  Activating project at `~/Repos/Heisenberg-Model-3D/Packages`


In [2]:
using DelimitedFiles 
using LinearAlgebra
using Statistics
using Random
using Printf
using Plots;

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423


In [33]:
function RandomSpin()
    """Creates a random unit vector"""
    θ = rand(0:1e-6:π)
    ϕ = rand(0:1e-6:2π)
    return [sin(θ)*cos(ϕ), sin(θ)*sin(ϕ), cos(θ)]
end

function AcceptanceProbability(Spin1, Spin2, n, β)
    """Computes the probability of adding Spin2 - neighbour of
    Spin1 - to the cluster. It uses the temperature β and the 
    vector of reflection n"""
    return 1 - exp(min(0,-2β*transpose(Spin1)*n*transpose(Spin2)*n))
end

function ReflectSpin(S,n,state)
    """Reflectes the spin state[S] around the plane defined by the
    normal vector n"""
    state[S] -=  2(transpose(state[S])*n)*n
end

function GetNeighbours(L,L2,L3)
    """Gets the coordinates of the 6 neighbours of every spin in the state. It uses 
    periodic boundery conditions in a 3D lattice of length L (area L2, volume L3). 
    The L3 × 6 matriz it returns contains the neighbours as folows: 
    [Up-Down-Left-Right-Front-Back]"""
    Neighbours = zeros(Int16,L3,6)
    for x in 1:L3
        Neighbours[x,:] = [L2*((x-1)÷L2)+(x-1-L+L2)%L2+1,
            L2*((x-1)÷L2)+(x-1+L)%L2+1,
            L*((x-1)÷L)+(x+L-2)%L+1,
            L*((x-1)÷L)+(x)%L+1,
            (x-L2+L3-1)%L3+1,
            (x+L2-1)%L3+1]
    end
    return Neighbours
end

function Grow_Reflect(S, Cluster, n, β, L, L2, L3, state, neighbours)
    """Checks every neighbour of spin state[S] and adds them to the Cluster given
        a certain probability (refair to AcceptanceProbability). Finally, it 
        reflects the spin around the normal vector n (refair to ReflectSpin)"""
    #Checks neighbours
    for Sn in neighbours[S,:]
        if Sn ∉ Cluster && rand(0:1e-15:1) < AcceptanceProbability(state[S],state[Sn],n,β)
            #Adds neighbour to Cluster
            push!(Cluster,Sn)
            #Checks neighbours of neighbour
            Grow_Reflect(Sn, Cluster, n, β, L, L2, L3, state, neighbours)
        end
    end
    #Reflects the spin
    ReflectSpin(S,n, state)
end

function NewState(state, L, L2, L3, β, neighbours)
    """Chooses a randon spin from the state with lattice sice L, and also a
    random normal vector. From there it builds the cluster using Wolff algorithm
    (see Grow_Reflect)"""
    n = RandomSpin()     #Initial random Normal Vector
    S0 = rand(1:1:L3)    #Initial random Spin
    Cluster = [S0]       #Stores the indexes of the spins added to the cluster
    Grow_Reflect(S0, Cluster, n, β, L, L2, L3, state, neighbours) #Builds cluster and flips for new state
end;

In [10]:
function Magnetization(state)
    """Returns the magnetization as the norm of the sum of all spins in the grid"""
    return norm(sum(state))
end;

function Energy(state,L,L2,L3,neighbours)
    """Returns the energy of a state"""
    energy = 0.0
    #Goes through every spin in the lattice
    for ii in 1:L3  
        Sp = [0,0,0]
        #Only checks 3 out the 6 neighbourds to avoid double counting of bounderies
        for Sn in neighbours[ii,[2,4,6]] 
           Sp += state[Sn]
        end
        energy -= dot(Sp,state[ii])
    end
    return energy
end;

function Variance(X)
    """Returns the variance of any array X"""
    return mean(X.^2) - mean(X)^2
end;

function Suceptibility(M,β)
    """Magnetic suceptibility as the variance of magnetization over tempeture"""
    return Variance(M)*β
end

function SpecificHeat(E,β)
    """Specific heat as the variance of energy over tempeture square"""
    return Variance(E)*β^2
end;

In [11]:
function Proceed(state, β, epochs, jumps, L, L2, L3, neighbours)
    """Creates a sequence of states and saves to a file once every fixed number of epochs 
     given by jumps. This is done for a single value of β in a lattice of size L."""
    for jj in 1:epochs
        NewState(state, L, L2, L3, β, neighbours)
        if jj%jumps == 0
            open("T_"*string(β)*".txt", "a") do io
                writedlm(io, state)
            end
        end
    end
end;

In [36]:
"""----- MAIN -----"""
L = 10  #Lattice size
L2 = L*L   #Lattice Area
L3 = L*L2  #Lattice Volume
epochs = 50000   #Number of states generated
jumps = 1000     #Number of steps after which magnetization is calculated
#Random.seed!(51)
state = [RandomSpin() for i in 1:L3] #The 3D lattice is store as a 1D-Array
neighbours = GetNeighbours(L,L2,L3)  #Matrix with the indexes of the 6 adjacent spins of each in the lattice
Temperatures = [kk for kk in 0.5:.2:2.5 ]
for kk in 1 ./Temperatures           #Loop over all tempetures
    Proceed(state, kk, epochs, jumps, L, L2, L3, neighbours)
end

In [ ]:
"""---- Magnetization vs Temperature ----"""
plot(T[5:30],Ms[5:30],seriestype = :scatter)